In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

In [16]:
data = pd.read_csv('CFTrend_data.csv')
data

classes = data['Classification'] 
data = data.drop(['Company', 'Time', 'Classification'], axis = 1)

col_list = []
for col in data.columns:
    col_list.append(col)
    
for col in col_list:
    print(col)
    data[col] = data[col].astype(float) # Converting columns to floats
    print("Done")
    
# Normalising the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(data) 
data.loc[:,:] = scaled_values

data.insert(data.shape[1], 'Classification', classes)
data

Net Income
Done
Cash from Operations
Done
Cash from Investing
Done
Cash from Financing
Done
Net Change in Cash
Done
Levered Free Cash Flow
Done
Unlevered Free Cash Flow
Done
Free Cash Flow / Share
Done


,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Classification
0,0.480473,0.469643,0.508359,0.534067,0.606119,0.542375,0.546646,0.437802,0
1,0.481818,0.467858,0.505735,0.537491,0.606840,0.540983,0.545741,0.437786,0
2,0.481596,0.459018,0.491027,0.551337,0.606217,0.531175,0.536074,0.437411,1
3,0.481972,0.469780,0.520974,0.528007,0.610585,0.543266,0.548056,0.437738,1
4,0.480921,0.466917,0.503488,0.538586,0.605890,0.541717,0.546469,0.437794,1
5,0.480900,0.472130,0.502963,0.537991,0.606221,0.542226,0.546985,0.437969,0
6,0.483575,0.446902,0.509085,0.540846,0.605936,0.539428,0.544301,0.437681,1
7,0.483352,0.470616,0.503060,0.537277,0.605155,0.542431,0.547178,0.437757,1
8,0.481091,0.459393,0.503048,0.539351,0.604342,0.538613,0.543393,0.437593,1
9,0.481496,0.472081,0.501630,0.551333,0.618644,0.541420,0.546175,0.437752,1


In [17]:
# Converting Data to Numpy Arrays
NpMatrix = data.to_numpy(dtype = None, copy = False)
X = NpMatrix[:,0:8] # Parameters
y = NpMatrix[:,8] # Price
print("X:", X)
print("\n")
print("Type X:", type(X))
print("Length of Individual X_train Vector:", len(X[1]))
print("Total Number of Training instances:", len(X))
print("\n")
print(y)
print("\n")
print("Type y:", type(y))
print("Length of Individual y_train vector", y[0])
print("Total number of y values", len(y))

X: [[0.48047315 0.46964303 0.5083592  ... 0.54237545 0.54664551 0.43780199]
 [0.48181768 0.46785821 0.50573495 ... 0.54098301 0.54574138 0.43778649]
 [0.48159606 0.45901847 0.49102705 ... 0.53117487 0.5360744  0.43741059]
 ...
 [0.47248322 0.38463906 0.47025539 ... 0.56701213 0.57357241 0.43757012]
 [0.48147547 0.46795254 0.49489803 ... 0.54175474 0.5464481  0.43774903]
 [0.50298102 0.46586734 0.50960056 ... 0.54332166 0.54786444 0.43779489]]


Type X: <class 'numpy.ndarray'>
Length of Individual X_train Vector: 8
Total Number of Training instances: 4500


[0. 0. 1. ... 1. 1. 1.]


Type y: <class 'numpy.ndarray'>
Length of Individual y_train vector 0.0
Total number of y values 4500


In [19]:
# Takes 1hour, 37 mins to run

# Using GridSearchCV to find best SVM hyperparameters
params = {'C': [0.1, 1, 10, 50, 100, 1000, 10000],
          'gamma': ['scale', 'auto'],# Slack parameter, how much error is allowed in prediction
          #[1,0.1,0.01,0.001], # Determines whether only close points determine hyperplane or those further away
          'kernel': ['linear', 'rbf', 'poly'],
         'degree' : [2,3]} #rnel used
 
cv_str = StratifiedKFold(n_splits=5)
SVM = SVC()

gs_modelSVM = GridSearchCV(SVM, params, cv= cv_str)
grid_results = gs_modelSVM.fit(X , y)

# Best parameters, best test score using these parameters, optimal KNN Configuration
print("Best Parameters", gs_modelSVM.best_params_)
print("Best score on Test Data", gs_modelSVM.best_score_)
print("Optimal Configuration", gs_modelSVM.best_estimator_)

Best Parameters {'C': 10, 'degree': 3, 'gamma': 'scale', 'kernel': 'poly'}
Best score on Test Data 0.6806666666666666
Optimal Configuration SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='poly',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [20]:
# Summarize the results in a readable format
print("Best: {0}, using {1}".format(grid_results.cv_results_['mean_test_score'], grid_results.best_params_))
results_df = pd.DataFrame(grid_results.cv_results_)
results_df

Best: [0.67511111 0.67511111 0.67422222 0.67511111 0.67511111 0.67511111
 0.67511111 0.67511111 0.67844444 0.67511111 0.67511111 0.67511111
 0.67511111 0.67644444 0.67444444 0.67511111 0.67511111 0.67511111
 0.67511111 0.67644444 0.67733333 0.67511111 0.67511111 0.67511111
 0.67511111 0.67711111 0.67688889 0.67511111 0.67511111 0.67511111
 0.67511111 0.67711111 0.68066667 0.67511111 0.67511111 0.67511111
 0.67511111 0.67555556 0.67666667 0.67511111 0.67511111 0.67511111
 0.67511111 0.67555556 0.67577778 0.67511111 0.67511111 0.67511111
 0.67511111 0.674      0.67622222 0.67511111 0.67511111 0.67511111
 0.67511111 0.674      0.67777778 0.67511111 0.67511111 0.67511111
 0.67511111 0.67422222 0.67755556 0.67511111 0.67444444 0.67488889
 0.67511111 0.67422222 0.678      0.67511111 0.67444444 0.67488889
 0.67488889 0.67044444 0.67822222 0.67488889 0.67422222 0.67444444
 0.67488889 0.67044444 0.68       0.67488889 0.67422222 0.67466667], using {'C': 10, 'degree': 3, 'gamma': 'scale', 'kernel

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_degree,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.148620,0.025487,0.016828,0.001613,0.1,2,scale,linear,"{'C': 0.1, 'degree': 2, 'gamma': 'scale', 'ker...",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,19
1,0.587868,0.171586,0.082959,0.019948,0.1,2,scale,rbf,"{'C': 0.1, 'degree': 2, 'gamma': 'scale', 'ker...",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,19
2,3.013163,1.006851,0.027198,0.004662,0.1,2,scale,poly,"{'C': 0.1, 'degree': 2, 'gamma': 'scale', 'ker...",0.673696,0.674806,0.674444,0.672970,0.675195,0.674222,0.000797,76
3,0.153116,0.025274,0.020799,0.002994,0.1,2,auto,linear,"{'C': 0.1, 'degree': 2, 'gamma': 'auto', 'kern...",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,19
4,0.391614,0.062341,0.054630,0.008007,0.1,2,auto,rbf,"{'C': 0.1, 'degree': 2, 'gamma': 'auto', 'kern...",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,19
5,0.233600,0.053124,0.040803,0.012744,0.1,2,auto,poly,"{'C': 0.1, 'degree': 2, 'gamma': 'auto', 'kern...",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,19
6,0.257832,0.015554,0.026401,0.005987,0.1,3,scale,linear,"{'C': 0.1, 'degree': 3, 'gamma': 'scale', 'ker...",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,19
7,0.363993,0.028873,0.040210,0.000421,0.1,3,scale,rbf,"{'C': 0.1, 'degree': 3, 'gamma': 'scale', 'ker...",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,19
8,21.058689,1.674886,0.034046,0.018158,0.1,3,scale,poly,"{'C': 0.1, 'degree': 3, 'gamma': 'scale', 'ker...",0.677026,0.680355,0.674444,0.685206,0.675195,0.678444,0.003947,3
9,0.133725,0.011839,0.016800,0.001600,0.1,3,auto,linear,"{'C': 0.1, 'degree': 3, 'gamma': 'auto', 'kern...",0.674806,0.674806,0.675556,0.675195,0.675195,0.675111,0.000282,19


In [21]:
results_df = results_df.sort_values(['rank_test_score'])
results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_degree,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
32,116.872670,19.644437,0.031192,0.009263,10,3,scale,poly,"{'C': 10, 'degree': 3, 'gamma': 'scale', 'kern...",0.682575,0.682575,0.678889,0.677419,0.681869,0.680667,0.002118,1
80,154.982501,47.653304,0.021146,0.001989,10000,3,scale,poly,"{'C': 10000, 'degree': 3, 'gamma': 'scale', 'k...",0.682575,0.678135,0.683333,0.676307,0.679644,0.680000,0.002645,2
8,21.058689,1.674886,0.034046,0.018158,0.1,3,scale,poly,"{'C': 0.1, 'degree': 3, 'gamma': 'scale', 'ker...",0.677026,0.680355,0.674444,0.685206,0.675195,0.678444,0.003947,3
74,100.907010,40.989001,0.020148,0.002469,10000,2,scale,poly,"{'C': 10000, 'degree': 2, 'gamma': 'scale', 'k...",0.680355,0.684795,0.672222,0.675195,0.678532,0.678222,0.004314,4
68,113.680578,13.327693,0.019558,0.001729,1000,3,scale,poly,"{'C': 1000, 'degree': 3, 'gamma': 'scale', 'ke...",0.680355,0.679245,0.675556,0.677419,0.677419,0.678000,0.001659,5
56,107.737834,17.604549,0.019755,0.001151,100,3,scale,poly,"{'C': 100, 'degree': 3, 'gamma': 'scale', 'ker...",0.680355,0.677026,0.675556,0.674082,0.681869,0.677778,0.002916,6
62,65.941540,17.380653,0.020553,0.001004,1000,2,scale,poly,"{'C': 1000, 'degree': 2, 'gamma': 'scale', 'ke...",0.680355,0.684795,0.671111,0.675195,0.676307,0.677556,0.004668,7
20,44.144342,5.002552,0.032903,0.012619,1,3,scale,poly,"{'C': 1, 'degree': 3, 'gamma': 'scale', 'kerne...",0.683685,0.677026,0.673333,0.676307,0.676307,0.677333,0.003423,8
25,0.640718,0.029688,0.045214,0.009404,10,2,scale,rbf,"{'C': 10, 'degree': 2, 'gamma': 'scale', 'kern...",0.678135,0.679245,0.673333,0.678532,0.676307,0.677111,0.002123,9
31,0.677538,0.080187,0.046404,0.001950,10,3,scale,rbf,"{'C': 10, 'degree': 3, 'gamma': 'scale', 'kern...",0.678135,0.679245,0.673333,0.678532,0.676307,0.677111,0.002123,9
